Model For Text Classification 

## Setup Mount Drive,Install And Import All Package

In [3]:
# Mounting data in Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import re
import nltk
import string
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
print("TF Version : {}".format(tf.__version__))

TF Version : 2.5.0


In [38]:
!pip install pickle5

     |████████████████████████████████| 133kB 5.2MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219280 sha256=324cc885d021be1725ea0098678c6a1ac3fe7d80451a4360e5d5bd2450170a6a
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Download Dataset From Github

In [44]:
URL_PATH = 'https://github.com/Capstone-B21-CAP0220/Dataset/blob/main/DatasetSAPPA.zip?raw=true'
df = pd.read_csv(URL_PATH, compression='zip', sep=',', quotechar='"')
df.head(3)

,No,Nama Pelapor,Jenis Pelapor,Lokasi,Deskripsi Laporan,Jenis kekerasan,Penanganan
0,1.0,Tina Isna,Korban,Semarang,"Aku anak tunggal. Ibu dan ayah berpisah ketika umur saya 4 tahun. Ibu menikah dengan laki-laki yang perkerjaannya notaris. Aku dipaksa ibu memanggil ayah baru dengan sebutan ""daddy"". Sejak saat itu juga daddy memperlakukanku tidak enak. Aku nggak ingat pas umur berapa, tapi aku udah diperkosa daddy selama bertahun-tahun. Aku juga disiksa sebelum aku diperkosa. Ibu enggak tau kejadian ini karena daddy punya kekuasaan tinggi di rumah. Tolong bantu aku, aku tidak tau harus mengadu kesiapa",Seksual,"Korban: Rehabilitas, Kasus: Penegak Hukum"
1,2.0,Dina Setiyawi,Korban,Makassar,"Saya sudah tidak tahan lagi. Dosen saya kurang ajar. Awalnya, saya dan 5 teman saya dianggap memiliki penyakit psikis, sehingga saya dan teman2 saya ditawarkan pengobatan dengan metode hipnoterapi atau spiritual oleh dosen saya. Dari awal saya sudah tidak nyaman, tetapi demi kesembuhan psikis saya, saya hanya diam aja. Kami diminta datang ke laboratorium mikrofarmasi, tapi rasanya tindakan dosen saya seperti tidak mengobati tetapi malah seperti mencabuli. Saya dipeluk erat, saya dicium dan saya diminta untuk memegang penisnya. Ini terjadi sudah hampir dua tahun, dan saya tidak tahan lagi.",Seksual,"Korban: Rehabilitas, Kasus: Penegak Hukum"
2,3.0,Nur Hayati,Orang tua korban,Bogor,"Saya ingin melapor. Anak saya dicabuli oleh ustadz di pondok pesantrennya. Saya menangis ketika anak saya berani menceritakannya kepada saya. Bagaimana saya terima ketika anak saya sedang tidur, pak ustadz meraba-raba tubuh anak saya. Dan kejinya, pas anak saya bangun, pak ustadz malah menggesek-gesekkan penisnya ke bagian vital anak saya sampai maaf spermanya keluar. Anak saya nggak berani lapor karena diancam tidak akan diluluskan. Dan ternyata ini nggak hanya terjadi kepada anak saya. Saya tidak terima. Saya ingin menuntut keadilan.",Seksual,"Korban: Rehabilitas, Kasus: Penegak Hukum"


## Preprocessing Data

### Cleaning Data

In [45]:
data = df[['Deskripsi Laporan', 'Jenis kekerasan']]
pd.set_option('display.max_colwidth', -1)
data.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Deskripsi Laporan,Jenis kekerasan
0,"Aku anak tunggal. Ibu dan ayah berpisah ketika umur saya 4 tahun. Ibu menikah dengan laki-laki yang perkerjaannya notaris. Aku dipaksa ibu memanggil ayah baru dengan sebutan ""daddy"". Sejak saat itu juga daddy memperlakukanku tidak enak. Aku nggak ingat pas umur berapa, tapi aku udah diperkosa daddy selama bertahun-tahun. Aku juga disiksa sebelum aku diperkosa. Ibu enggak tau kejadian ini karena daddy punya kekuasaan tinggi di rumah. Tolong bantu aku, aku tidak tau harus mengadu kesiapa",Seksual
1,"Saya sudah tidak tahan lagi. Dosen saya kurang ajar. Awalnya, saya dan 5 teman saya dianggap memiliki penyakit psikis, sehingga saya dan teman2 saya ditawarkan pengobatan dengan metode hipnoterapi atau spiritual oleh dosen saya. Dari awal saya sudah tidak nyaman, tetapi demi kesembuhan psikis saya, saya hanya diam aja. Kami diminta datang ke laboratorium mikrofarmasi, tapi rasanya tindakan dosen saya seperti tidak mengobati tetapi malah seperti mencabuli. Saya dipeluk erat, saya dicium dan saya diminta untuk memegang penisnya. Ini terjadi sudah hampir dua tahun, dan saya tidak tahan lagi.",Seksual
2,"Saya ingin melapor. Anak saya dicabuli oleh ustadz di pondok pesantrennya. Saya menangis ketika anak saya berani menceritakannya kepada saya. Bagaimana saya terima ketika anak saya sedang tidur, pak ustadz meraba-raba tubuh anak saya. Dan kejinya, pas anak saya bangun, pak ustadz malah menggesek-gesekkan penisnya ke bagian vital anak saya sampai maaf spermanya keluar. Anak saya nggak berani lapor karena diancam tidak akan diluluskan. Dan ternyata ini nggak hanya terjadi kepada anak saya. Saya tidak terima. Saya ingin menuntut keadilan.",Seksual


In [7]:
def clean_text(text):
  #mengubah semua karakter huruf menjadi huruf kecil
  text = text.lower()
  #menghilangkan nama akun
  text = re.sub('@[^\s]+', '', text)
  #menghilangkan punctuation
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  #menghilangkan angka
  text = re.sub('\w*\d\w*', '', text)
  #menghilangkan URL
  text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
  text = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?\S','', text)
  #menghilangkan hashtag
  text = re.sub('#[^\s]+', '', text)
  #menghilangkan huruf tunggal
  text = re.sub(r'\b[a-zA-Z]\b', '', text)
  return text

clean = lambda x: clean_text(x)

data['Deskripsi Laporan'] = data['Deskripsi Laporan'].apply(clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
def get_stopword(stopwordfile):
  stopwords = []
  file_stopwords = open(stopwordfile, 'r')
  row = file_stopwords.readline()
  while row:
    word = row.strip()
    stopwords.append(word)
    row = file_stopwords.readline()
  file_stopwords.close()
  return stopwords


stop_words_indo = get_stopword('/content/drive/MyDrive/Capstone B21-CAP0220/Progress ML/stopwordsindo.txt')

In [16]:
def stopwords(text):
  tokens = word_tokenize(text)
  filtered = []

  for w in tokens:
    if w not in stop_words_indo:
      filtered.append(w)
  
  hasil = ' '.join(filtered)
  return hasil

st = lambda x: stopwords(x)

In [19]:
data['Deskripsi Laporan'] = pd.DataFrame(data['Deskripsi Laporan'].apply(st))
data.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Deskripsi Laporan,Jenis kekerasan
0,anak tunggal ibu ayah berpisah umur ibu menikah lakilaki perkerjaannya notaris dipaksa ibu memanggil ayah sebutan daddy daddy memperlakukanku tidak enak nggak pas umur udah diperkosa daddy bertahuntahun disiksa diperkosa ibu enggak tau kejadian daddy kekuasaan rumah tolong bantu tidak tau mengadu kesiapa,Seksual
1,tidak tahan dosen ajar teman dianggap memiliki penyakit psikis ditawarkan pengobatan metode hipnoterapi spiritual dosen tidak nyaman kesembuhan psikis diam aja laboratorium mikrofarmasi tindakan dosen tidak mengobati mencabuli dipeluk erat dicium memegang penisnya tidak tahan,Seksual
2,melapor anak dicabuli ustadz pondok pesantrennya menangis anak berani menceritakannya terima anak tidur ustadz merabaraba tubuh anak kejinya pas anak bangun ustadz menggesekgesekkan penisnya vital anak maaf spermanya anak nggak berani lapor diancam tidak diluluskan nggak anak tidak terima menuntut keadilan,Seksual


### Change Label

In [20]:
df = data[['Deskripsi Laporan', 'Jenis kekerasan']]
df = df.rename(columns ={'Deskripsi Laporan': 'sentence', 'Jenis kekerasan': 'label'}, inplace=False)
df.head(1)

,sentence,label
0,anak tunggal ibu ayah berpisah umur ibu menikah lakilaki perkerjaannya notaris dipaksa ibu memanggil ayah sebutan daddy daddy memperlakukanku tidak enak nggak pas umur udah diperkosa daddy bertahuntahun disiksa diperkosa ibu enggak tau kejadian daddy kekuasaan rumah tolong bantu tidak tau mengadu kesiapa,Seksual


In [21]:
class_names = ['Seksual', 'Trafiking', 'Migran', 'Fisik', 'Psikis', 'Ekonomi']

def change_value(input_value):
  '''Change label from categorical to numeric'''
  for index, label in enumerate(class_names):
    if input_value == label:
      return index 
  return -1

df['label'] = data['Jenis kekerasan'].apply(change_value)
df.tail()

,sentence,label
185,diporoti pacar,5
186,uang habis pacar membeli kebutuhannya,5
187,pacar nama berhutang judi dikejar kejar tagihan,5
188,ibu dikejar rentenir akibat ulang ayah meminjam uang judi,5
189,dipaksa memenuhi kehidupan ekonomi pacar,5


In [22]:
# class_names = ['Seksual', 'Trafiking', 'Migran', 'Fisik', 'Psikis', 'Ekonomi']

df_seksual = df[df['label'] == 0]
df_trafiking = df[df['label'] == 1]
df_migran = df[df['label'] == 2]
df_fisik = df[df['label'] == 3]
df_psikis = df[df['label'] == 4]
df_ekonomi = df[df['label'] == 5]

df_train_seksual = df_seksual.sample(30)
df_train_trafiking = df_trafiking.sample(30)
df_train_migran = df_migran.sample(23)
df_train_fisik = df_fisik.sample(30)
df_train_psikis = df_psikis.sample(30)
df_train_ekonomi = df_ekonomi.sample(30)

df_seksual_remaining = df_seksual.drop(df_train_seksual.index)
df_trafiking_remaining = df_trafiking.drop(df_train_trafiking.index)
df_migran_remaining = df_migran.drop(df_train_migran.index)
df_fisik_remaining = df_fisik.drop(df_train_fisik.index)
df_psikis_remaining = df_psikis.drop(df_train_psikis.index)
df_ekonomi_remaining = df_ekonomi.drop(df_train_ekonomi.index)

df_validate_seksual = df_seksual_remaining.sample(3)
df_validate_trafiking = df_trafiking_remaining.sample(3)
df_validate_migran = df_migran_remaining.sample(2)
df_validate_fisik = df_fisik_remaining.sample(3)
df_validate_psikis = df_psikis_remaining.sample(3)
df_validate_ekonomi = df_ekonomi_remaining.sample(3)

# df_test_seksual = df_seksual_remaining.drop(df_validate_seksual.index)
# df_test_trafiking = df_trafiking_remaining.drop(df_validate_trafiking.index)
# df_test_migran = df_migran_remaining.drop(df_validate_migran.index)
# df_test_fisik = df_fisik_remaining.drop(df_validate_fisik.index)
# df_test_psikis = df_psikis_remaining.drop(df_validate_psikis.index)
# df_test_ekonomi = df_ekonomi_remaining.drop(df_validate_ekonomi.index)


train_df = pd.concat([df_train_seksual, df_train_trafiking, df_train_migran, df_train_fisik, df_train_psikis, df_train_ekonomi]).sample(frac=1)
validate_df = pd.concat([df_validate_seksual, df_validate_trafiking, df_validate_migran, df_validate_fisik, df_validate_psikis, df_validate_ekonomi]).sample(frac=1)
test_df = validate_df

print("Untuk data train, validate dan test sudah seimbang")
print("Jumlah data train : {}".format(len(train_df)))
print("Jumlah data validate : {}".format(len(validate_df)))
print("Jumlah data tes : {}".format(len(test_df)))

Untuk data train, validate dan test sudah seimbang
Jumlah data train : 173
Jumlah data validate : 17
Jumlah data tes : 17


In [23]:
train_sentences = train_df['sentence'].values
validation_sentences = validate_df['sentence'].values
train_labels = train_df['label'].values
validation_labels = validate_df['label'].values

jumlah1 = len(train_sentences)
jumlah2 = len(train_labels)
jumlah3 = len(validation_sentences)
jumlah4 = len(validation_labels)

print("Train Sentences : {}\nTrain Labels: {}\nValidation Sentences : {}\nValidation Labels : {}".format(jumlah1,jumlah2,jumlah3,jumlah4))

Train Sentences : 173
Train Labels: 173
Validation Sentences : 17
Validation Labels : 17


In [24]:
train_sentences[:3]

array(['memiliki adik perempuan dimana adiksaya memiliki pacar adiksaya diajak jalan jalan pacarnya dikeluarga kegitan tidak ditinggalkan adik bilang adiksaya tidak pacarnya ngotot pacarnya pulang kerumah adiksaya kasar tampar mengalami lukak memar',
       'uang habis pacar membeli kebutuhannya', 'dipaksa orang tua tidak'],
      dtype=object)

In [25]:
train_labels[:3]

array([3, 5, 4])

### Tokenization

In [26]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [27]:
data.head(2)

,Deskripsi Laporan,Jenis kekerasan
0,anak tunggal ibu ayah berpisah umur ibu menikah lakilaki perkerjaannya notaris dipaksa ibu memanggil ayah sebutan daddy daddy memperlakukanku tidak enak nggak pas umur udah diperkosa daddy bertahuntahun disiksa diperkosa ibu enggak tau kejadian daddy kekuasaan rumah tolong bantu tidak tau mengadu kesiapa,Seksual
1,tidak tahan dosen ajar teman dianggap memiliki penyakit psikis ditawarkan pengobatan metode hipnoterapi spiritual dosen tidak nyaman kesembuhan psikis diam aja laboratorium mikrofarmasi tindakan dosen tidak mengobati mencabuli dipeluk erat dicium memegang penisnya tidak tahan,Seksual


In [29]:
labels = data['Jenis kekerasan'].values
labels


# Ubah train label 
train_labels_str = [class_names[label] for label in train_labels]
validation_labels_str = [class_names[label] for label in validation_labels]

In [30]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizer For Sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)



word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

print("Total word : {}".format(len(word_index)))

print('\nDictionary list :')
print(word_index)

print('\nSequences data pertama')
print(train_sequences[0])
print('\nPadding data pertama')
print(train_padded[0])
print("\nTottal padding : {}".format(len(train_padded[0])))

# Tokenizer For Label
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)



training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels_str))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels_str))

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

Total word : 1226

Dictionary list :
{'<OOV>': 1, 'tidak': 2, 'anak': 3, 'teman': 4, 'ibu': 5, 'orang': 6, 'ayah': 7, 'rumah': 8, 'pacar': 9, 'pulang': 10, 'uang': 11, 'memiliki': 12, 'tau': 13, 'tolong': 14, 'suami': 15, 'temanku': 16, 'dipaksa': 17, 'pacarnya': 18, 'pekerjaan': 19, 'ku': 20, 'sekolah': 21, 'perempuan': 22, 'dimana': 23, 'pekerja': 24, 'malam': 25, 'nggak': 26, 'bantu': 27, 'mencoba': 28, 'takut': 29, 'majikan': 30, 'tua': 31, 'kejadian': 32, 'hutang': 33, 'keluarga': 34, 'tetangga': 35, 'paman': 36, 'bilang': 37, 'marah': 38, 'kemarin': 39, 'tertekan': 40, 'migran': 41, 'luka': 42, 'negeri': 43, 'disiksa': 44, 'disana': 45, 'bercerita': 46, 'hubungan': 47, 'mengikuti': 48, 'lakilaki': 49, 'memukul': 50, 'dosen': 51, 'kali': 52, 'seksual': 53, 'tangan': 54, 'kerja': 55, 'dijual': 56, 'udah': 57, 'kekerasan': 58, 'mendengar': 59, 'melapor': 60, 'menangis': 61, 'minggu': 62, 'langsung': 63, 'intim': 64, 'barang': 65, 'mencari': 66, 'memaksa': 67, 'mengancam': 68, 'tugas

## Build And Train Model

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               2176      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 175       
Total params: 21,447
Trainable params: 21,447
Non-trainable params: 0
_________________________________________________________________


In [33]:
num_epochs = 50
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq))

Epoch 1/50
6/6 [==============================] - 1s 42ms/step - loss: 1.9432 - accuracy: 0.1792 - val_loss: 1.9375 - val_accuracy: 0.1765
Epoch 2/50
6/6 [==============================] - 0s 9ms/step - loss: 1.9351 - accuracy: 0.1734 - val_loss: 1.9297 - val_accuracy: 0.1765
Epoch 3/50
6/6 [==============================] - 0s 10ms/step - loss: 1.9275 - accuracy: 0.1734 - val_loss: 1.9219 - val_accuracy: 0.1765
Epoch 4/50
6/6 [==============================] - 0s 9ms/step - loss: 1.9191 - accuracy: 0.1734 - val_loss: 1.9132 - val_accuracy: 0.1765
Epoch 5/50
6/6 [==============================] - 0s 9ms/step - loss: 1.9109 - accuracy: 0.1734 - val_loss: 1.9039 - val_accuracy: 0.1765
Epoch 6/50
6/6 [==============================] - 0s 10ms/step - loss: 1.9007 - accuracy: 0.1734 - val_loss: 1.8938 - val_accuracy: 0.1765
Epoch 7/50
6/6 [==============================] - 0s 8ms/step - loss: 1.8902 - accuracy: 0.1734 - val_loss: 1.8831 - val_accuracy: 0.1765
Epoch 8/50
6/6 [===============

## Model ussage

In [34]:
new_sentence = ['Pacar saya selalu menghukum saya dan tidak boleh mengizinkan saya keluar bermain dengan teman saya']
new_sequences = tokenizer.texts_to_sequences(new_sentence)
new_padded = pad_sequences(new_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(new_padded))

[[0.00096517 0.19527426 0.17377359 0.1695943  0.1829824  0.15502469
  0.12238562]]


### Saved Model

In [35]:
from keras.models import load_model
model.save('/content/drive/MyDrive/Capstone B21-CAP0220/Progress ML/my_model.h5') 

In [36]:
os.listdir('/content/drive/MyDrive/Capstone B21-CAP0220/Progress ML')

['Capstone.ipynb',
 'stopwordsindo.txt',
 'DatasetSAPPA.csv',
 'DatasetSAPPA.zip',
 'my_model.h5']

### Load Model

In [40]:
pickle.dump(tokenizer, open('/content/drive/MyDrive/Capstone B21-CAP0220/Progress ML/tokenizer.pkl', 'wb'))

In [42]:
os.listdir('/content/drive/MyDrive/Capstone B21-CAP0220/Progress ML')

['Capstone.ipynb',
 'stopwordsindo.txt',
 'DatasetSAPPA.csv',
 'DatasetSAPPA.zip',
 'my_model.h5',
 'tokenizer.pkl']

Rsult: 
- Model : my_model.h5
- Tokenizer : tokenizer.pkl